# Template Matching

Similarity Metrics for Template Matching:
- SSD
- SAD
- NCC

<a>
  <img src="../images/matching_metrices.png" alt="Pinhole camera" width="500" >
</a>

# Feature & Descriptor Matching